In [14]:
from facenet_pytorch import InceptionResnetV1,MTCNN

model= InceptionResnetV1(pretrained='vggface2').eval()  # Set to evaluation mode

In [15]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from torch.nn.functional import cosine_similarity


In [16]:
base_path = "../face_match_aiic_r2/face-matching/images"

def load_image(image_id):
    image_path = os.path.join(base_path, image_id + ".jpg")
    img = Image.open(image_path).convert('RGB')
    return img

load_image("001").show()  # Display the image with ID "001"

In [17]:
total_images = len(glob.glob(os.path.join(base_path, "*.jpg")))
print(f"Total images found: {total_images}")

Total images found: 109


In [27]:
all_ids = [f"{i:03d}" for i in range(total_images)]
print(all_ids)

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108']


In [18]:
ref_df = pd.read_csv("../face_match_aiic_r2/face-matching/ref_img.csv")
ref_ids = [f"{int(x):03d}" for x in ref_df.iloc[:, 0]]
print(ref_ids)

['048', '025', '095', '043', '105', '071', '046', '096', '020', '085', '061', '073', '084', '026', '008']


In [20]:
mtcnn=MTCNN(
    image_size=160,
    margin=14,
    min_face_size=40,
    
)

In [21]:
@torch.no_grad()
def extract_embedding(image_id):
    img = load_image(image_id)
    face = mtcnn(img)
    if face is not None:
        face = face.unsqueeze(0)  # Add batch dimension
        embedding = model(face)
        embedding = embedding / embedding.norm()  # Normalize the embedding
        return embedding.squeeze(0)  # Remove batch dimension
    else:
        return None

In [28]:
embeddings=[]
embedding_ids=[]

for img_id in tqdm(all_ids):
    emb = extract_embedding(img_id)
    if emb is not None:
        embeddings.append(emb)
        embedding_ids.append(img_id)
    
embeddings = np.stack(embeddings)  # shape: [N, 512]

id_to_index = {img_id: idx for idx, img_id in enumerate(embedding_ids)} # to map image id to index in embeddings array. ex - '000' -> 0 


100%|██████████| 109/109 [00:46<00:00,  2.34it/s]


In [30]:
def cosine_sim(ref_emb, all_embs):
    return np.dot(all_embs, ref_emb)

In [31]:
MIN_SIM = 0.60   # similarity threshold 

results = []

for ref_id in ref_ids:
    ref_idx = id_to_index[ref_id]
    ref_emb = embeddings[ref_idx]

    # Compute cosine similarity with all images
    sims = np.dot(embeddings, ref_emb)   # shape: [N]

    # Select indices above threshold
    matched_idx = np.where(sims >= MIN_SIM)[0]

    # Convert indices to image IDs
    matched_ids = [embedding_ids[i] for i in matched_idx]

    # Ensure reference image is included
    if ref_id not in matched_ids:
        matched_ids.append(ref_id)

    # Sort IDs numerically (important for clean submission)
    matched_ids = sorted(matched_ids)

    results.append({
        "ref_img": ref_id,
        "photos": "|".join(matched_ids)
    })


In [33]:
submission = pd.DataFrame(results, columns=["ref_img", "photos"])

submission.to_csv("submission.csv", index=False)

print("Submission file saved as:", "submission.csv")
submission.head()


Submission file saved as: submission.csv


,ref_img,photos
0,048,048|078|101|102
1,025,005|006|025|032|033|040|041|042|067|083|086|094
2,095,014|021|045|047|072|095|097
3,043,013|023|027|034|043|100
4,105,001|024|050|105
